In [ ]:
# Installing the packages
!pip install rdflib pyshacl

In [ ]:
# Loading the functions

from rdflib import Graph
from pyshacl import validate

# Function to load RDF data from a string
def load_rdf_data(rdf_data_str, format='turtle'):
    g = Graph()
    g.parse(data=rdf_data_str, format=format)
    return g

# Function to load SHACL shapes from a string
def load_shacl_shapes(shacl_shapes_str, format='turtle'):
    shapes_g = Graph()
    shapes_g.parse(data=shacl_shapes_str, format=format)
    return shapes_g

# Function to validate RDF data against SHACL shapes
def validate_rdf_data(rdf_data_graph, shacl_shapes_graph):
    conforms, results_graph, results_text = validate(
        data_graph=rdf_data_graph,
        shacl_graph=shacl_shapes_graph,
        inference='rdfs',  # Optional: you can use 'rdfs', 'owlrl', or 'none'
        abort_on_first=False,
        allow_warnings=True,
        meta_shacl=False,
        advanced=True,
        js=False,
        debug=False
    )
    return conforms, results_text


In [ ]:
# Shapes

shacl_shapes = """
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix ex: <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:PersonShape
    a sh:NodeShape ;
    sh:targetClass ex:Person ;
    sh:property [
        sh:path ex:age ;
        sh:datatype xsd:integer ;
        sh:maxCount 1 ;
    ] ;
    sh:property [
        sh:path ex:email ;
        sh:datatype xsd:string ;
        # sh:pattern "^.+@.+\\..+$" ;
        sh:pattern "^.+@.+\\\\..+$" ;
    ] .
    
"""

# Data

rdf_data = """
@prefix ex: <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:John a ex:Person ;
        ex:age "28" ;
        ex:email "john@example.org" .
        
"""

# Load RDF data and SHACL shapes
rdf_graph = load_rdf_data(rdf_data, format='turtle')
shacl_graph = load_shacl_shapes(shacl_shapes, format='turtle')

# Validate the RDF data
conforms, results_text = validate_rdf_data(rdf_graph, shacl_graph)

# Output the results
if conforms:
    print("RDF data conforms to SHACL shapes.")
else:
    print("RDF data does NOT conform to SHACL shapes.")
    
print(results_text)

# 1. Simple example from the SHACL specification

In [ ]:
# Shapes

shacl_shapes = """
@prefix ex: <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sdo: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:PersonShape
	a sh:NodeShape ;
	sh:targetClass ex:Person ;    # Applies to all persons
	sh:property [                 # _:b1
		sh:path ex:ssn ;           # constrains the values of ex:ssn
		sh:maxCount 1 ;
		sh:datatype xsd:string ;
		sh:pattern "^\\\\d{3}-\\\\d{2}-\\\\d{4}$" ;
	] ;
	sh:property [                 # _:b2
		sh:path ex:worksFor ;
		sh:class ex:Company ;
		sh:nodeKind sh:IRI ;
	] ;
	sh:closed true ;
	sh:ignoredProperties ( rdf:type ) .
    
"""

# Data

rdf_data = """
@prefix ex: <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sdo: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Alice
	a ex:Person ;
	ex:ssn "987-65-432A" .
  
ex:Bob
	a ex:Person ;
	ex:ssn "123-45-6789" ;
	ex:ssn "124-35-6789" .
  
ex:Calvin
	a ex:Person ;
	ex:birthDate "1971-07-07"^^xsd:date ;
	ex:worksFor ex:UntypedCompany .

"""

# Load RDF data and SHACL shapes
rdf_graph = load_rdf_data(rdf_data, format='turtle')
shacl_graph = load_shacl_shapes(shacl_shapes, format='turtle')

# Validate the RDF data
conforms, results_text = validate_rdf_data(rdf_graph, shacl_graph)

# Output the results
if conforms:
    print("RDF data conforms to SHACL shapes.")
else:
    print("RDF data does NOT conform to SHACL shapes.")
    
print(results_text)

# 2. Chapter 5 from Validating RDF

## 5.1. Simple Example - Example 95  UserShape example in SHACL

In [ ]:
# Shapes

shacl_shapes = """
@prefix : <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:UserShape a sh:NodeShape;
   sh:targetClass :User ;
   sh:property [                  # Blank node 1
    sh:path     schema:name ;
    sh:minCount 1;
    sh:maxCount 1;
    sh:datatype xsd:string ;
  ] ;
  sh:property [                   # Blank node 2
   sh:path schema:gender ;
   sh:minCount 1;
   sh:maxCount 1;
   sh:or (
    [ sh:in (schema:Male schema:Female) ]
    [ sh:datatype xsd:string]
   )
  ] ;
  sh:property [                   # Blank node 3  
   sh:path     schema:birthDate ;
   sh:maxCount 1;
   sh:datatype xsd:date ;
  ] ;
  sh:property [                   # Blank node 4 
   sh:path     schema:knows ;
   sh:nodeKind sh:IRI ;
   sh:class    :User ;
  ] .
  
"""

# Data

rdf_data = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:alice a :User;                             #Passes as a :UserShape     
       schema:name           "Alice" ;
       schema:gender         schema:Female ;
       schema:knows          :bob .

:bob   a :User;                             #Passes as a :UserShape     
       schema:gender         schema:Male ;
       schema:name           "Robert";
       schema:birthDate      "1980-03-10"^^xsd:date .

:carol a :User;                             #Passes as a :UserShape     
       schema:name           "Carol" ;
       schema:gender         schema:Female ;
       foaf:name             "Carol" .
"""

# Load RDF data and SHACL shapes
rdf_graph = load_rdf_data(rdf_data, format='turtle')
shacl_graph = load_shacl_shapes(shacl_shapes, format='turtle')

# Validate the RDF data
conforms, results_text = validate_rdf_data(rdf_graph, shacl_graph)

# Output the results
if conforms:
    print("RDF data conforms to SHACL shapes.")
else:
    print("RDF data does NOT conform to SHACL shapes.")
    
print(results_text)

## 5.6.1  Node shapes

In [ ]:
# Shapes

shacl_shapes = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:UserShape a              sh:NodeShape;
           sh:targetClass :User ;
           sh:nodeKind    sh:IRI .
"""

# Data

rdf_data = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:alice a :User .                     #Passes as a :UserShape 

<http://other.uri.com/bob> a :User . #Passes as a :UserShape 

_:1 a :User .                        #Fails as a :UserShape 

""" 

# Load RDF data and SHACL shapes
rdf_graph = load_rdf_data(rdf_data, format='turtle')
shacl_graph = load_shacl_shapes(shacl_shapes, format='turtle')

# Validate the RDF data
conforms, results_text = validate_rdf_data(rdf_graph, shacl_graph)

# Output the results
if conforms:
    print("RDF data conforms to SHACL shapes.")
else:
    print("RDF data does NOT conform to SHACL shapes.")
    
print(results_text)

## 5.6.2. Property Shapes

In [ ]:
# Shapes

shacl_shapes = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:UserShape a sh:NodeShape;
 sh:targetClass :User ;
 sh:property [
   sh:path     [sh:alternativePath (schema:knows schema:follows)] ;
   sh:nodeKind sh:IRI ;
   sh:minCount 1
 ] ;
 sh:property [
   sh:path     ([sh:oneOrMorePath schema:knows] schema:email) ;
   sh:nodeKind sh:IRI
 ].
 
"""

# Data

rdf_data = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:alice a              :User ;                 #Passes as a :UserShape
       schema:follows <mailto:alice@mail.org>;
       schema:knows   :bob, :carol .

:bob   schema:email <mailto:bob@mail.org>;
       schema:knows :carol .

:carol schema:email <mailto:carol@mail.org> .

:dave  a :User ;                              #Fails as a :UserShape
       schema:knows <mailto:dave@mail.org> ;
       schema:knows :carol, :emily .

:emily schema:email "Unknown" .

""" 


# Load RDF data and SHACL shapes
rdf_graph = load_rdf_data(rdf_data, format='turtle')
shacl_graph = load_shacl_shapes(shacl_shapes, format='turtle')

# Validate the RDF data
conforms, results_text = validate_rdf_data(rdf_graph, shacl_graph)

# Output the results
if conforms:
    print("RDF data conforms to SHACL shapes.")
else:
    print("RDF data does NOT conform to SHACL shapes.")
    
print(results_text)


## 5.7.4. Target subjects of

In [ ]:
# Shapes

shacl_shapes = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:UserShape a sh:NodeShape;
 sh:targetSubjectsOf :teaches ;
 sh:property [
    sh:path schema:name ;
    sh:minCount 1;
    sh:maxCount 1;
    sh:datatype xsd:string ;
 ] .
 
"""

# Data

rdf_data = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:alice :teaches :Algebra ;     #Passes as a :UserShape
       schema:name "Alice" .

:bob   :teaches :Logic ;       #Fails as a :UserShape
       foaf:name "Robert" .

:carol foaf:name 23 .          # Ignored

""" 


# Load RDF data and SHACL shapes
rdf_graph = load_rdf_data(rdf_data, format='turtle')
shacl_graph = load_shacl_shapes(shacl_shapes, format='turtle')

# Validate the RDF data
conforms, results_text = validate_rdf_data(rdf_graph, shacl_graph)

# Output the results
if conforms:
    print("RDF data conforms to SHACL shapes.")
else:
    print("RDF data does NOT conform to SHACL shapes.")
    
print(results_text)

## 5.9.1. Datatypes

In [ ]:
# Shapes

shacl_shapes = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:UserShape a sh:NodeShape ;
  sh:targetClass :User ;
  sh:property [
    sh:path     schema:name ;
    sh:datatype xsd:string
  ] ;
  sh:property [
    sh:path     schema:birthDate ;
    sh:datatype xsd:date ;
  ] .
 
"""

# Data

rdf_data = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .


:alice a :User ;                                 #  Passes as a :User   
       schema:name "Alice";
       schema:birthDate "1981-07-10"^^xsd:date .

:bob   a :User  ;                                #  Fails as a :User  
       schema:name "Robert" ;
       schema:birthDate 1981 .

:carol a :User ;                                 #  Fails as a :User
       schema:name :Carol ;
       schema:birthDate "2003-06-10"^^xsd:date .

:dave  a :User ;                                 #  Fails as a :User
       schema:name "Dave" ;
       schema:birthDate "Unknown"^^xsd:date .
       
""" 


# Load RDF data and SHACL shapes
rdf_graph = load_rdf_data(rdf_data, format='turtle')
shacl_graph = load_shacl_shapes(shacl_shapes, format='turtle')

# Validate the RDF data
conforms, results_text = validate_rdf_data(rdf_graph, shacl_graph)

# Output the results
if conforms:
    print("RDF data conforms to SHACL shapes.")
else:
    print("RDF data does NOT conform to SHACL shapes.")
    
print(results_text)

## 5.9.4.  Sets of Values

In [ ]:
# Shapes

shacl_shapes = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:UserShape a sh:NodeShape ;
  sh:targetClass :User ;
  sh:property [
   sh:path schema:gender ;
   sh:in   ( schema:Male schema:Female )
  ] .
 
"""

# Data

rdf_data = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .


:alice a                  :User;        #Passes as a :UserShape 
       schema:affiliation :OurCompany ;
       schema:gender      schema:Female .

:bob   a                  :User;        #Fails as a :UserShape 
       schema:gender      schema:male .
       
""" 


# Load RDF data and SHACL shapes
rdf_graph = load_rdf_data(rdf_data, format='turtle')
shacl_graph = load_shacl_shapes(shacl_shapes, format='turtle')

# Validate the RDF data
conforms, results_text = validate_rdf_data(rdf_graph, shacl_graph)

# Output the results
if conforms:
    print("RDF data conforms to SHACL shapes.")
else:
    print("RDF data does NOT conform to SHACL shapes.")
    
print(results_text)

## 5.9.5.  Specific Value

In [ ]:
# Shapes

shacl_shapes = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:UserShape a sh:NodeShape ;
  sh:targetClass :User ;
  sh:property [
   sh:path     schema:affiliation ;
   sh:hasValue :OurCompany
  ] .
 
"""

# Data

rdf_data = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .


:alice a :User;                              #Passes as a :UserShape 
       schema:affiliation :OurCompany .

:bob   a :User;                              #Fails as a :UserShape 
       schema:affiliation :OurUniversity .

:carol a :User .                             #Fails as a :UserShape 

:dave  a :User;                              #Passes as a :UserShape 
       schema:affiliation :OurCompany ;
       schema:affiliation :OurUniversity .
       
""" 


# Load RDF data and SHACL shapes
rdf_graph = load_rdf_data(rdf_data, format='turtle')
shacl_graph = load_shacl_shapes(shacl_shapes, format='turtle')

# Validate the RDF data
conforms, results_text = validate_rdf_data(rdf_graph, shacl_graph)

# Output the results
if conforms:
    print("RDF data conforms to SHACL shapes.")
else:
    print("RDF data does NOT conform to SHACL shapes.")
    
print(results_text)

# 3. Let's create our own example

In [ ]:
# Shapes

shacl_shapes = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

# Create shapes here
 
"""

# Data

rdf_data = """
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .


# Create data here
       
""" 


# Load RDF data and SHACL shapes
rdf_graph = load_rdf_data(rdf_data, format='turtle')
shacl_graph = load_shacl_shapes(shacl_shapes, format='turtle')

# Validate the RDF data
conforms, results_text = validate_rdf_data(rdf_graph, shacl_graph)

# Output the results
if conforms:
    print("RDF data conforms to SHACL shapes.")
else:
    print("RDF data does NOT conform to SHACL shapes.")
    
print(results_text)